In [2]:
%pip install -q "ultralytics>=8.3.117" cython filterpy scikit-image \
               opencv-python-headless==4.10.0.84 tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into 

In [3]:
import sys, torch, ultralytics as ul
print("Py:", sys.version)
print("Torch:", torch.__version__)
print("Ultralytics:", ul.__version__)

from ultralytics import YOLO
import cv2, torch, time, pathlib, collections, numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Py: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Torch: 2.5.1+cu124
Ultralytics: 8.3.133


In [4]:
import cv2
import time
import torch
from ultralytics import YOLO

# ───── CONFIG ─────
INPUT_VIDEO  = "/kaggle/input/video1/WhatsApp Video 2025-05-13 at 22.03.19.mp4"  # your 20 s clip
OUTPUT_VIDEO = "output_with_counts.mp4"
MISS_TOL     = 30   # frames tolerance before declaring MISSING (~1 s @ 30 FPS)

# choose GPU if available
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

# ───── MODEL LOADING ─────
model = YOLO("yolov8n.pt")
model.fuse()          # in-place fuse Conv+BN
model.to(DEVICE)
if DEVICE.startswith("cuda"):
    model.model.half()

# ───── INVENTORY FOR NEW / MISSING EVENTS ─────
class LiveInventory:
    def __init__(self, miss_tolerance=30):
        self.last_seen = {}     # id → last frame index
        self.frame_idx = 0
        self.miss_tol  = miss_tolerance

    def update(self, tracks):
        current = {int(t.id) for t in tracks if t.id is not None}

        # NEW = seen this frame but never before
        new_ids = [i for i in current if i not in self.last_seen]

        # refresh timestamps
        for i in current:
            self.last_seen[i] = self.frame_idx

        # MISSING = dropped out beyond tolerance
        missing = [i for i, last in list(self.last_seen.items())
                   if self.frame_idx - last > self.miss_tol]
        for i in missing:
            del self.last_seen[i]

        self.frame_idx += 1
        return new_ids, missing

# ───── PROCESSING LOOP ─────
cap = cv2.VideoCapture(INPUT_VIDEO)
fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
w, h   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out    = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps_in, (w, h))

inv         = LiveInventory(miss_tolerance=MISS_TOL)
entered_cnt = 0
exited_cnt  = 0
start       = time.time()
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # track with YOLOv8 + ByteTrack
    results = model.track(
        frame,
        tracker="bytetrack.yaml",
        persist=True,
        device=DEVICE,
        imgsz=640,
        verbose=False
    )
    tracks = results[0].boxes

    # compute events
    new_ids, missing_ids = inv.update(tracks)
    entered_cnt += len(new_ids)
    exited_cnt  += len(missing_ids)

    # draw boxes+IDs
    vis = results[0].plot()

    # overlay the running counters
    text = f"Entered: {entered_cnt}   Exited: {exited_cnt}"
    cv2.putText(vis, text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (0, 255, 0), 2, cv2.LINE_AA)

    # (optional) also draw latest event IDs below
    evt_text = f"New IDs: {new_ids}  Missing IDs: {missing_ids}"
    cv2.putText(vis, evt_text, (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                (200, 200, 200), 1, cv2.LINE_AA)

    out.write(vis)
    frame_count += 1

# cleanup
cap.release()
out.release()
elapsed = time.time() - start
print(f"Processed {frame_count} frames in {elapsed:.1f}s → {frame_count/elapsed:.2f} FPS")


YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
Processed 1239 frames in 20.6s → 60.22 FPS


In [5]:
import zipfile
from IPython.display import FileLink, display

# Create a zip archive containing your output video
with zipfile.ZipFile('outputnew.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('/kaggle/working/output_with_counts.mp4')

# Display a download link for the zip file
display(FileLink('outputnew.zip'))


/kaggle/working/outputnew.zip